# GAN
- 이미지에서는 VAE보다 GAN이 대세

In [ ]:
import tensorflow as tf
import matplotlibe.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

In [ ]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [ ]:
# 128 x 256
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
# 128 x 784
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [ ]:
# GAN에는 생성기와 판별기가 있다
# cost function : 확률점 함수를 사용
# Kulback-libler??   KL-Divergence 함수를 사용  -> GAN, VAE에서 사용
#       -> 분포의 차를 확인하는 함수
#       -> 생성기 : nosie가 input data [cost function : KL-Divergence]
#       -> 

# 그동안은 MSE(예측), Enthropy(분류) 사용했었다.

def generator(noise_z): # 128 x 128
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1) # 128x256
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2) # 128x784   784 이미지 생성
    return output 

def discriminator(inputs):# 실제이미지의 분포를 확인
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise)

In [ ]:
G = generator(Z) # 노이즈 => 이미지 생성
D_gene = discriminator(G) # 생성되어진 이미지 판별함[분포를 확인함]
D_real = discriminator(X) # 실제이미지의 분포를 확인

# log 함수 붙은 이유? 확률값에 로그 취하는 이유는? 정보량을 구하는 것
# 확률이 높아지면 정보량이 작아지고,
# 확률이 낮아지면 정보량이 커짐\
# 마치 tf idf의 논리와 같.
# 가장 적합한 분포를 찾아가는 것이니 높은 것이 유리하다.
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene)# 최대우도 추정법 : 적합한 분포가 (확률이 높은) 될 수 있도록
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]
                        
# 최대화하기 위해 -를 줌
train_D = tf.train.AdamOptimizer(learning_rate).minimize( -loss_D, var_list=D_var_list) # 역전파 변수를 지정
train_G = tf.train.AdamOptimizer(learning_rate).minimize( -loss_G, var_list=G_var_list)

In [ ]:
%matplotlib inline
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [ ]:
# 이미지 분포의 특징을 추출하는 망
for epoch in range(total_epoch):
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)    
        # 실제 이미지 분포
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X: batch_xs, Z : noise})     
        # 생성된 이미지 분포
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict ={Z:noise})
        
    print('Epoch :', '%04d' %epoch,
         'D loss :', '{:.4}'.format(loss_val_D),
         'G loss :', '{:.4}'.format(loss_val_G))    
    
    if epoch==0 or (epoch + 1) % 10 == 0:
        sample_size = 10

        # 노이즈로부터 이미지를 생성 'generator'
        noise = get_noise(sample_size, n_noise) # 10x128
        samples = sess.run(G, feed_dict = {Z:noise})
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1)) 
        
        for i in range(sample_size) :
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28,28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print("최적화완료")